In [1]:
import pandas as pd
pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", None)
import tkinter as tk
from tkinter import filedialog

In [2]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [3]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [6]:
iris = df[["BG", "Subcategory", "Group", "狀態", "預交日期", "負責業務", "品名", "本國幣別NTD"]]

In [12]:
iris = iris[(iris["BG"] == "RF") & (iris["狀態"].str.contains("出")) & (iris["預交日期"].between("2021-02-01", "2021-02-28"))]

In [16]:
iris["Subcategory"] = iris.apply(lambda x: "BU2" if "RFDPA" in x["品名"] else "BU1", axis=1)

<ipython-input-16-4981040ffd7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iris["Subcategory"] = iris.apply(lambda x: "BU2" if "RFDPA" in x["品名"] else "BU1", axis=1)


In [30]:
result = iris.pivot_table(index=["負責業務", "Subcategory"], values="本國幣別NTD", aggfunc="sum", margins=True, margins_name="Total").reset_index()

In [32]:
result.style.format({"本國幣別NTD":"{0:,.0f}"}).highlight_max(color="lightgreen").highlight_min(color="red")

,負責業務,Subcategory,本國幣別NTD
0,墨漢雷迪,BU1,"12,872,583"
1,許凱智,BU1,"6,835,328"
2,許凱智,BU2,"3,783,283"
3,鄭里緗,BU1,"191,101"
4,Total,,"23,682,295"
